# 1.test fastai

In [ ]:
import fastai
from fastai import *
from fastai.text.all import *
from fastai.callback import *

In [2]:
path = untar_data(URLs.IMDB_SAMPLE)

In [3]:
path.ls()

(#2) [Path('C:/Users/home/.fastai/data/imdb_sample/models'),Path('C:/Users/home/.fastai/data/imdb_sample/texts.csv')]

In [4]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False
2,negative,"Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt li...",False
3,positive,"Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man's life - interestingly enough the man's entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" are not just mere words blathered from the lips of a high-brassed offic...",False
4,negative,"This movie succeeds at being one of the most unique movies you've seen. However this comes from the fact that you can't make heads or tails of this mess. It almost seems as a series of challenges set up to determine whether or not you are willing to walk out of the movie and give up the money you just paid. If you don't want to feel slighted you'll sit through this horrible film and develop a real sense of pity for the actors involved, they've all seen better days, but then you realize they actually got paid quite a bit of money to do this and you'll lose pity for them just like you've alr...",False


In [5]:
df['text'][1]

'This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is some merit in this view, but it\'s also true that no one forced Hindus and Muslims in the region to mistreat each other as they did around the time of partition. It seems more likely that the British simply saw the tensions between the religions and were clever enough to exploit them to their own ends.<br /><br />The result is that there is much cruelty and inhumanity in the situation and this is very u

In [6]:
from fastai.text import *
# Tokenization และ Numericalization
databunch_languagemodel = TextDataLoaders.from_csv(path=path, csv_fname='texts.csv')

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck


f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\data\transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)


In [7]:
databunch_languagemodel.show_batch(max_n=3)

f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\data\transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)


,text,category
0,xxbos negative,"**Possible Spoilers**<br /><br />This straight-to-video mess combines a gun-toting heroine, grade-Z effects, nazis, a mummy and endless lesbian footage and it's still boring; the video's 45 minute running time only SEEMS like Eternity.The only good part is one of the blooper outtakes, wherein the bad guys force a 400-pound Egyptologist into a chair--and one villain's foot almost gets crushed under a chair leg. Take this snoozer back to the video store and watch televised golf, bowling or tennis instead."
1,xxbos positive,"For a low budget project, the Film was a success. The story is interesting, and the actors were convincing. Eva Longoria, who now stars on the TV Show ""Dragnet,"" is sexier than ever. The locations were ideal for the ganster plot, and the director shows his talent by taking on many roles for his project. Of course this low budget film could use better editing transitions and more special effects for the gun scenes, but the music keeps this script moving. Although this film has it's share of problems, such as continuity, I must say that I would rent the director's next movie. If your a film student, you could learn a few things from the director's commentary."
2,xxbos positive,"...........as I was when I saw this movie) I will never watch this movie again, not because it is a bad movie, but because it scared me so much! As I said, I was 14 when my English teacher decided to show it to us; the reason for this is that we had read an extract from the book.<br /><br />All the girls in my class were TERRIFIED when the Woman in Black comes through the window and floats over Kidd's bed, although, just before that there is something that also frightened us, which was when Kidd finds the toy soldier underneath his pillow, and he hears a child's voice say ""It's for you"". That scene still haunts me to this day, nearly 7 YEARS after I saw the film.<br /><br />If you are easily scared, AVOID THIS FILM!!!!!!!!"


In [ ]:

dls = TextDataLoaders.from_csv(
    path, 
    "texts.csv", 
    is_lm=True, 
    valid_pct=0.1,  # สัดส่วนข้อมูลสำหรับชุดตรวจสอบ
    seq_len=72      # ความยาวของลำดับ (BPTT)
)

print(dls.one_batch())

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck
(LMTensorText([[ 2,  9,  2,  ...,  9,  2, 10],
              [ 2, 10,  2,  ...,  9,  2,  9],
              [ 2,  9,  2,  ...,  9,  2,  9],
              ...,
              [ 2, 10,  2,  ...,  9,  2, 10],
              [ 2,  9,  2,  ..., 10,  2,  9],
              [ 2, 10,  2,  ..., 10,  2, 10]]), TensorText([[ 9,  2,  9,  ...,  2, 10,  2],
            [10,  2, 10,  ...,  2,  9,  2],
            [ 9,  2,  9,  ...,  2,  9,  2],
            ...,
            [10,  2,  9,  ...,  2, 10,  2],
            [ 9,  2,  9,  ...,  2,  9,  2],
            [10,  2,  9,  ...,  2, 10,  2]]))


In [22]:
x, y = dls.train.one_batch()
example_batch_size = 15 
example_seq_len = 12
decoded_tuples = [dls.decode(tens.tolist()) for tens in x] 

formatted_data = []
for i, item_tuple in enumerate(decoded_tuples):
    text_obj = item_tuple[0]  # เลือก Input [0]

    string_data = repr(text_obj) 
    
    if string_data.startswith("'") and string_data.endswith("'"):
        string_data = string_data[1:-1]
        
    formatted_line = f"{i}{string_data.replace(' ', '')}"
    formatted_data.append(formatted_line)

texts_df = pd.DataFrame(formatted_data)
texts_df.columns = [0] 

print(texts_df)

                    0
0    0LMTensorText(2)
1    1LMTensorText(2)
2    2LMTensorText(2)
3    3LMTensorText(2)
4    4LMTensorText(2)
..                ...
59  59LMTensorText(2)
60  60LMTensorText(2)
61  61LMTensorText(2)
62  62LMTensorText(2)
63  63LMTensorText(2)

[64 rows x 1 columns]


In [23]:
"""
สร้าง language_model_learner ด้วย AWD-LSTM เป็นโมเดลแบบ RNN, LSTM ไว้เราจะอธิบายต่อไป

AWD_LSTM เป็น Language Model ที่ได้ถูกเทรนมากับ Corpus ขนาดใหญ่ จากส่วนหนึ่งของ Wikipedia ชื่อ WikiText-103 มีขนาดใหญ่ถึง 103,227,021 Token และ 267,735 Vocab เรียบร้อยแล้ว

กำหนดค่า drop_mult (Dropout) เรื่อง Regularization ไว้เราจะอธิบายต่อไป
"""
databunch_languagemodel = TextDataLoaders.from_csv(path, 'texts.csv', text_col='text', is_lm=True, valid_pct=0.1)
learn = language_model_learner(databunch_languagemodel, AWD_LSTM, drop_mult=0.3, 
                               metrics=[accuracy, Perplexity()]).to_fp16()

# เทรนรอบแรก (Freeze)
learn.fit_one_cycle(1, 2e-2)

# Save
learn.save('1epoch')

# เทรนต่อ (Unfreeze)
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

# Save Encoder (สำคัญ! เพื่อเอาไปใช้ต่อในขั้นตอน Classifier)
learn.save_encoder('finetuned')

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck


f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\callback\fp16.py:47: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  self.autocast,self.learn.scaler,self.scales = autocast('cuda', dtype=dtype),GradScaler('cuda', **self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.097510,4.036071,0.271562,56.603500,02:44


f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\callback\fp16.py:47: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  self.autocast,self.learn.scaler,self.scales = autocast('cuda', dtype=dtype),GradScaler('cuda', **self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.658293,3.990862,0.276104,54.101498,02:59
1,3.599629,3.991494,0.276677,54.135693,02:59
2,3.435856,4.021513,0.271407,55.785473,02:58
3,3.143050,4.096679,0.266199,60.140213,02:59
4,2.783891,4.229847,0.260804,68.706718,02:59
5,2.391639,4.393215,0.253348,80.900108,03:02


KeyboardInterrupt: 

In [ ]:
import warnings

# ใส่บรรทัดนี้ไว้ส่วนบนสุดของไฟล์ หรือก่อนจุดที่เกิด warning
warnings.filterwarnings("ignore")
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2

preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)]
print("\n".join(preds))

f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\callback\fp16.py:47: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  self.autocast,self.learn.scaler,self.scales = autocast('cuda', dtype=dtype),GradScaler('cuda', **self.kwargs),L()
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\callback\fp16.py:47: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  self.autocast,self.learn.scaler,self.scales = autocast('cuda', dtype=dtype),GradScaler('cuda', **self.kwargs),L()
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\

f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\callback\fp16.py:47: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  self.autocast,self.learn.scaler,self.scales = autocast('cuda', dtype=dtype),GradScaler('cuda', **self.kwargs),L()
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\callback\fp16.py:47: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  self.autocast,self.learn.scaler,self.scales = autocast('cuda', dtype=dtype),GradScaler('cuda', **self.kwargs),L()
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\

i liked this movie because the characters are well - developed and very developed . The characters are very developed and have a great deal of time . The friendship is such a bloom and i will never forget anyone when i watch
i liked this movie because it 's like someone got a bunch of things to say about it , especially the idea that it is just not a type of movie like the John Lennon Story . Before you watch it


# 2.sklearn

In [57]:
import pandas as pd
from pythainlp.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [31]:
data = [
    ("อาหารอร่อยมาก บริการดี", "positive"),
    ("ชอบมากครับ ส่งของไว", "positive"),
    ("เยี่ยมมาก", "positive"),
    ("ชอบมาก", "positive"),
    ("สินค้าคุณภาพแย่ ไม่คุ้มราคา", "negative"),
    ("รอนานมาก ผิดหวัง", "negative"),
    ("เฉยๆ นะ พอกินได้", "neutral"),
    ("ก็โอเคครับ ตามราคา", "neutral")
]
df = pd.DataFrame(data, columns=['text', 'sentiment'])

# 2. ฟังก์ชันตัดคำภาษาไทย (Tokenization)
# ผลลัพธ์จะเป็นลิสต์ของคำ เช่น "อาหารอร่อยมาก" -> "อาหาร อร่อย มาก"
def thai_tokenizer(text):
    return word_tokenize(text, engine="newmm")

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], test_size=0.2, random_state=42)

In [58]:
# 
model_naive_bayes = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=thai_tokenizer)),
    ('clf', MultinomialNB())
])
model_svm = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=thai_tokenizer)),
    ('clf', LinearSVC(dual='auto'))
])
model_rf = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=thai_tokenizer)),
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))
])
model = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=thai_tokenizer, ngram_range=(1,2))), 
    ('clf', LogisticRegression(solver='lbfgs'))
])

In [47]:
model

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidf', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",<function tha...00285BAE354E0>


In [ ]:
model.fit(X_train, y_train)
model_naive_bayes.fit(X_train, y_train)
model_rf.fit(X_train, y_train)
model_svm.fit(X_train, y_train)

f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidf', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",<function tha...00285BAE35440>


In [60]:
# test_text = ["บริการแย่มาก ไม่ประทับใจเลย"]
test_text = ["เยี่ยมมาก สุดยอดเลยครับ"]
prediction = model.predict(test_text)
prediction_svm = model_svm.predict(test_text)
prediction_nb = model_naive_bayes.predict(test_text)
prediction_rf = model_rf.predict(test_text)

print(f"ประโยค: {test_text[0]}")
print(f"ผลลัพธ์: {prediction[0]}")
print(f"ผลลัพธ์: {prediction_svm[0]}")
print(f"ผลลัพธ์: {prediction_nb[0]}")
print(f"ผลลัพธ์: {prediction_rf[0]}")

ประโยค: เยี่ยมมาก สุดยอดเลยครับ
ผลลัพธ์: positive
ผลลัพธ์: positive
ผลลัพธ์: positive
ผลลัพธ์: positive


In [61]:
y_pred = model.predict(X_test)
y_pred_rf = model_rf.predict(X_test)
y_pred_svm = model_svm.predict(X_test)
y_pred_nb = model_naive_bayes.predict(X_test)

# 7. แสดงผลลัพธ์
print("\n📊 Evaluation Report:")
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_nb))


📊 Evaluation Report:
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         1
    positive       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         1
    positive       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         1
    positive       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         

f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
f

In [52]:
vectorizer = model.named_steps['tfidf']
classifier = model.named_steps['clf']
feature_names = vectorizer.get_feature_names_out()

coefs = classifier.coef_[0] # ดึง weight ของ class แรก
print(coefs)
print(vectorizer)
print(classifier)
print(feature_names)

[ 0.01678656 -0.04616653 -0.03733573 -0.04037189 -0.03733573  0.18950537
 -0.03733573 -0.03733573 -0.04616653 -0.04616653 -0.04616653 -0.04616653
  0.18950537  0.18950537  0.18950537  0.18950537 -0.07718992 -0.07718992
 -0.04037189 -0.04616653 -0.04616653 -0.03733573 -0.03733573 -0.04037189
 -0.04037189 -0.03733573 -0.03733573 -0.13482904 -0.04037189  0.1175399
  0.18950537  0.18950537 -0.04037189 -0.04037189 -0.04037189 -0.04037189
 -0.03733573 -0.03733573 -0.07718992 -0.07718992  0.18950537  0.18950537
 -0.04616653 -0.04616653 -0.03733573  0.18950537  0.18950537 -0.03733573
 -0.03733573]
TfidfVectorizer(ngram_range=(1, 2),
                tokenizer=<function thai_tokenizer at 0x00000285BAE354E0>)
LogisticRegression()
[' ' '  ตาม' '  นะ' '  บริการ' '  พอ' '  ไม่' 'กิน' 'กิน ได้' 'ก็'
 'ก็ โอเค' 'ครับ' 'ครับ  ' 'คุณภาพ' 'คุณภาพ แย่' 'คุ้ม' 'คุ้ม ราคา' 'ชอบ'
 'ชอบ มาก' 'ดี' 'ตาม' 'ตาม ราคา' 'นะ' 'นะ  ' 'บริการ' 'บริการ ดี' 'พอ'
 'พอ กิน' 'มาก' 'มาก  ' 'ราคา' 'สินค้า' 'สินค้า คุณภาพ' 'อร

In [53]:
filename = 'sentiment_model.pkl'

with open(filename, 'wb') as file:
    pickle.dump(model, file)

print(f"\n💾 บันทึกโมเดลเรียบร้อยที่: {filename}")


💾 บันทึกโมเดลเรียบร้อยที่: sentiment_model.pkl


In [ ]:
import pickle
from pythainlp.tokenize import word_tokenize

def thai_tokenizer(text):
    return word_tokenize(text, engine="newmm")

with open('sentiment_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

new_texts = [
    "ร้านนี้บริการดีมาก แนะนำเลย",
    "เสียความรู้สึก รอนานเกิ้น",
    "เฉยๆ ก็พอกินได้"
]

predictions = loaded_model.predict(new_texts)
probs = loaded_model.predict_proba(new_texts) # ดูค่าความมั่นใจ (Optional)

print("\n🔮 ผลการทำนาย:")
for text, pred, prob in zip(new_texts, predictions, probs):
    confidence = max(prob) * 100
    print(f"ข้อความ: '{text}' -> ผลลัพธ์: {pred} (มั่นใจ {confidence:.2f}%)")


🔮 ผลการทำนาย:
ข้อความ: 'ร้านนี้บริการดีมาก แนะนำเลย' -> ผลลัพธ์: positive (มั่นใจ 50.29%)
ข้อความ: 'เสียความรู้สึก รอนานเกิ้น' -> ผลลัพธ์: positive (มั่นใจ 43.23%)
ข้อความ: 'เฉยๆ ก็พอกินได้' -> ผลลัพธ์: neutral (มั่นใจ 50.01%)


# Deep Learning 

In [63]:
import numpy as np
import pandas as pd
from pythainlp.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [64]:
data = [
    ("อาหาร อร่อย มาก", 1), ("บริการ ดี เยี่ยม", 1), ("ชอบ มาก ครับ", 1),
    ("สินค้า ดี ส่ง ไว", 1), ("คุ้ม ค่า มาก", 1), ("ประทับ ใจ สุดๆ", 1),
    ("รสชาติ แย่ มาก", 0), ("บริการ ช้า ไม่ ดี", 0), ("พนักงาน พูด ไม่ เพราะ", 0),
    ("ของ พัง เสียหาย", 0), ("ไม่ คุ้ม เงิน เลย", 0), ("ผิด หวัง มาก", 0)
] * 10

In [65]:
texts = [item[0] for item in data] # จริงๆ ต้องใช้ PyThaiNLP ตัดคำก่อนนะครับ แต่อันนี้ผมเว้นวรรคมาให้แล้ว
labels = np.array([item[1] for item in data])

In [66]:
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

In [67]:
max_length = 10
X_pad = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
y = labels

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

print(f"Original: {texts[0]}")
print(f"Sequence: {sequences[0]}")
print(f"Padded:   {X_pad[0]}")

Original: อาหาร อร่อย มาก
Sequence: [7, 8, 2]
Padded:   [7 8 2 0 0 0 0 0 0 0]


In [69]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Flatten, Conv1D

def build_model(model_type='lstm'):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=16, input_length=max_length))
    if model_type == 'mlp':
        model.add(Flatten()) 
        model.add(Dense(10, activation='relu'))
    elif model_type == 'cnn':
        model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
        model.add(GlobalMaxPooling1D()) # ดึงค่าที่เด่นที่สุดออกมา
    elif model_type == 'lstm':
        model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = build_model('lstm') 


history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=1)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 170ms/step - accuracy: 0.5000 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6929
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5000 - loss: 0.6923 - val_accuracy: 0.5000 - val_loss: 0.6919
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5000 - loss: 0.6914 - val_accuracy: 0.5000 - val_loss: 0.6906
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5000 - loss: 0.6895 - val_accuracy: 0.5833 - val_loss: 0.6889
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8438 - loss: 0.6879 - val_accuracy: 1.0000 - val_loss: 0.6867
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.9688 - loss: 0.6848 - val_accuracy: 1.0000 - val_loss: 0.6835
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 1.0000 - loss: 0.6805 - val_accuracy: 1.0000 - val_loss: 0.6788
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 0.6739 - val_accuracy: 1.0000 - val_loss: 0.6716

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(torch.from_numpy(X_train).long(), torch.from_numpy(y_train).float())
test_data = TensorDataset(torch.from_numpy(X_test).long(), torch.from_numpy(y_test).float())

batch_size = 16
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

class LSTMNet(nn.Module):
    def __init__(self, vocab_size, output_dim, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(LSTMNet, self).__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, (h0, c0))
        lstm_out = lstm_out[:, -1, :] 
        out = self.dropout(lstm_out)
        out = self.fc(out)
        sig_out = self.sigmoid(out)
        return sig_out

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMNet(vocab_size, output_dim=1, embedding_dim=16, hidden_dim=32, n_layers=1).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print("🚀 PyTorch Training Start...")
for epoch in range(10):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        model.zero_grad()
        output = model(inputs)
        loss = criterion(output.squeeze(), labels)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1} finished.")
print("✅ Training Complete")

f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


🚀 PyTorch Training Start...
Epoch 1 finished.
Epoch 2 finished.
Epoch 3 finished.
Epoch 4 finished.
Epoch 5 finished.
Epoch 6 finished.
Epoch 7 finished.
Epoch 8 finished.
Epoch 9 finished.
Epoch 10 finished.
✅ Training Complete


In [ ]:
import torch
from tensorflow.keras.preprocessing.sequence import pad_sequences #

input_text = "สินค้าคุ้มราคามาก ชอบครับ"

seq = tokenizer.texts_to_sequences([input_text]) 
pad = pad_sequences(seq, maxlen=10, padding='post', truncating='post')

input_tensor = torch.from_numpy(pad).long()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_tensor = input_tensor.to(device)
model.eval()

with torch.no_grad():
    output = model(input_tensor)
    probability = output.item()

print(f"ข้อความ: {input_text}")
print(f"ความน่าจะเป็น (Probability): {probability:.4f}")

if probability > 0.5:
    print("ผลลัพธ์: Positive (บวก) ✅")
else:
    print("ผลลัพธ์: Negative (ลบ) ❌")

ข้อความ: สินค้าคุ้มราคามาก ชอบครับ
ความน่าจะเป็น (Probability): 0.4902
ผลลัพธ์: Negative (ลบ) ❌
